# Stage 1: Data Preparation & Feast Setup

This notebook covers:
- Downloading Walmart sales dataset from Kaggle
- Feature engineering (lags, rolling stats) - **embedded in notebook**
- Registering features with Feast
- Testing feature retrieval

**Prerequisites:**
- Kaggle API credentials (environment variables recommended)
- Competition rules accepted: https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/rules

## 1. Install Dependencies

In [ ]:
%pip install feast==0.54.0 kaggle==1.7.4.5 pandas==2.2.3 pyarrow==17.0.0 scikit-learn==1.6.1 psycopg2-binary==2.9.11

In [1]:
import os
import pandas as pd
import numpy as np
import zipfile
from pathlib import Path
from feast import FeatureStore
import time

print('Imports successful !')

Imports successful !


## 2. Configure Kaggle Credentials

**Option 1: Environment Variables (Recommended)**

In [ ]:
# Set your Kaggle credentials (uncomment and fill in)
os.environ['KAGGLE_USERNAME'] = '<>'
os.environ['KAGGLE_KEY'] = '<>'

# Verify configuration
has_env = 'KAGGLE_USERNAME' in os.environ and 'KAGGLE_KEY' in os.environ
has_file = (Path.home() / '.kaggle' / 'kaggle.json').exists()

if has_env:
    print(f'Kaggle credentials configured (env vars)')
elif has_file:
    print(f'Kaggle credentials configured (file)')
else:
    print('Kaggle credentials not configured!')

Kaggle credentials configured (env vars)


## 3. Download Dataset from Kaggle

In [3]:
# Setup paths
data_dir = Path('/shared/datasets')
data_dir.mkdir(parents=True, exist_ok=True)

# Check if already downloaded
required_files = ['train.csv', 'features.csv', 'stores.csv']
files_exist = all((data_dir / f).exists() for f in required_files)

if files_exist:
    print('CSV files already exist')
else:
    print('Downloading from Kaggle...')
    
    from kaggle.api.kaggle_api_extended import KaggleApi
    
    # Initialize and authenticate
    api = KaggleApi()
    api.authenticate()
    
    # Download competition files
    competition = 'walmart-recruiting-store-sales-forecasting'
    api.competition_download_files(competition, path=str(data_dir))
    
    # Extract main zip
    main_zip = data_dir / f'{competition}.zip'
    if main_zip.exists():
        with zipfile.ZipFile(main_zip, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        main_zip.unlink()  # Remove zip after extraction
    
    # Extract individual CSV zips if needed
    for zip_name in ['train.csv.zip', 'features.csv.zip']:
        zip_path = data_dir / zip_name
        if zip_path.exists():
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(data_dir)
            zip_path.unlink()
    
    print('Download complete!')

# Verify files
print('\nDownloaded files:')
for file in sorted(data_dir.glob('*.csv')):
    size_mb = file.stat().st_size / 1024 / 1024
    print(f'   {file.name}: {size_mb:.1f} MB')

CSV files already exist

Downloaded files:
   features.csv: 0.6 MB
   stores.csv: 0.0 MB
   train.csv: 12.2 MB


## 4. Load and Explore Data

In [4]:
# Load CSV files
train_df = pd.read_csv(data_dir / 'train.csv')
features_df = pd.read_csv(data_dir / 'features.csv')
stores_df = pd.read_csv(data_dir / 'stores.csv')

# Convert dates
train_df['Date'] = pd.to_datetime(train_df['Date'])
features_df['Date'] = pd.to_datetime(features_df['Date'])

print('Dataset Summary:')
print(f'   Sales records: {len(train_df):,}')
print(f'   Stores: {train_df["Store"].nunique()}')
print(f'   Departments: {train_df["Dept"].nunique()}')
print(f'   Date range: {train_df["Date"].min().date()} to {train_df["Date"].max().date()}')
print(f'\nColumns:')
print(f'   train.csv: {list(train_df.columns)}')
print(f'   features.csv: {list(features_df.columns)}')
print(f'   stores.csv: {list(stores_df.columns)}')

train_df.head(3)

Dataset Summary:
   Sales records: 421,570
   Stores: 45
   Departments: 81
   Date range: 2010-02-05 to 2012-10-26

Columns:
   train.csv: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']
   features.csv: ['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday']
   stores.csv: ['Store', 'Type', 'Size']


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False


## 5. Feature Engineering: Sales Features

Compute time-series features (lags, rolling stats)

In [5]:
%%time

print('Computing time-series features...\n')

# Sort by store, dept, date
train_df = train_df.sort_values(['Store', 'Dept', 'Date'])

# Compute features for each store-dept combination
sales_list = []
total = train_df[['Store', 'Dept']].drop_duplicates().shape[0]

for idx, (store_dept, group) in enumerate(train_df.groupby(['Store', 'Dept']), 1):
    if idx % 500 == 0:
        print(f'   Processed {idx}/{total} store-dept combinations...')
    
    group = group.sort_values('Date').copy()
    
    # Lag features (previous weeks)
    group['sales_lag_1'] = group['Weekly_Sales'].shift(1)
    group['sales_lag_2'] = group['Weekly_Sales'].shift(2)
    group['sales_lag_4'] = group['Weekly_Sales'].shift(4)
    
    # Rolling statistics
    group['sales_rolling_mean_4'] = group['Weekly_Sales'].rolling(4, min_periods=1).mean()
    group['sales_rolling_mean_12'] = group['Weekly_Sales'].rolling(12, min_periods=1).mean()
    group['sales_rolling_std_4'] = group['Weekly_Sales'].rolling(4, min_periods=1).std().fillna(0)
    
    sales_list.append(group)

sales_df = pd.concat(sales_list, ignore_index=True)

# Rename to feast convention (lowercase)
sales_df = sales_df.rename(columns={
    'Store': 'store',
    'Dept': 'dept',
    'Date': 'date',
    'Weekly_Sales': 'weekly_sales',
    'IsHoliday': 'is_holiday'
})

# Select final columns
sales_df = sales_df[[
    'store', 'dept', 'date', 'weekly_sales', 'is_holiday',
    'sales_lag_1', 'sales_lag_2', 'sales_lag_4',
    'sales_rolling_mean_4', 'sales_rolling_mean_12', 'sales_rolling_std_4'
]]

print(f'\nSales features created: {sales_df.shape}')
sales_df.head(3)

Computing time-series features...

   Processed 500/3331 store-dept combinations...
   Processed 1000/3331 store-dept combinations...
   Processed 1500/3331 store-dept combinations...
   Processed 2000/3331 store-dept combinations...
   Processed 2500/3331 store-dept combinations...
   Processed 3000/3331 store-dept combinations...

Sales features created: (421570, 11)
CPU times: user 5.43 s, sys: 178 ms, total: 5.61 s
Wall time: 5.66 s


,store,dept,date,weekly_sales,is_holiday,sales_lag_1,sales_lag_2,sales_lag_4,sales_rolling_mean_4,sales_rolling_mean_12,sales_rolling_std_4
0,1,1,2010-02-05,24924.50,False,NaN,NaN,NaN,24924.500000,24924.500000,0.000000
1,1,1,2010-02-12,46039.49,True,24924.50,NaN,NaN,35481.995000,35481.995000,14930.552614
2,1,1,2010-02-19,41595.55,False,46039.49,24924.5,NaN,37519.846667,37519.846667,11131.900957


## 6. Feature Engineering: Store Features

Merge external factors with store metadata

In [6]:
print('Creating store features...\n')

# Merge features with stores
store_df = features_df.merge(stores_df, on='Store', how='left')

# Get unique store-dept combinations from train
unique_combos = train_df[['Store', 'Dept']].drop_duplicates()

# Expand store features to store+dept level
store_expanded = []
for _, row in unique_combos.iterrows():
    store_data = store_df[store_df['Store'] == row['Store']].copy()
    store_data['Dept'] = row['Dept']
    store_expanded.append(store_data)

store_expanded_df = pd.concat(store_expanded, ignore_index=True)

# Fill missing markdowns with 0
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
for col in markdown_cols:
    store_expanded_df[col] = store_expanded_df[col].fillna(0)

# Compute total markdown
store_expanded_df['total_markdown'] = store_expanded_df[markdown_cols].sum(axis=1)
store_expanded_df['has_markdown'] = (store_expanded_df['total_markdown'] > 0).astype(int)

# Rename to feast convention
store_expanded_df = store_expanded_df.rename(columns={
    'Store': 'store',
    'Dept': 'dept',
    'Date': 'date',
    'Temperature': 'temperature',
    'Fuel_Price': 'fuel_price',
    'CPI': 'cpi',
    'Unemployment': 'unemployment',
    'MarkDown1': 'markdown1',
    'MarkDown2': 'markdown2',
    'MarkDown3': 'markdown3',
    'MarkDown4': 'markdown4',
    'MarkDown5': 'markdown5',
    'Type': 'store_type',
    'Size': 'store_size'
})

# Select columns
store_expanded_df = store_expanded_df[[
    'store', 'dept', 'date', 'temperature', 'fuel_price', 'cpi', 'unemployment',
    'markdown1', 'markdown2', 'markdown3', 'markdown4', 'markdown5',
    'total_markdown', 'has_markdown', 'store_type', 'store_size'
]]

print(f'Store features created: {store_expanded_df.shape}')
store_expanded_df.head(3)

Creating store features...

Store features created: (606242, 16)


,store,dept,date,temperature,fuel_price,cpi,unemployment,markdown1,markdown2,markdown3,markdown4,markdown5,total_markdown,has_markdown,store_type,store_size
0,1,1,2010-02-05,42.31,2.572,211.096358,8.106,0.0,0.0,0.0,0.0,0.0,0.0,0,A,151315
1,1,1,2010-02-12,38.51,2.548,211.242170,8.106,0.0,0.0,0.0,0.0,0.0,0.0,0,A,151315
2,1,1,2010-02-19,39.93,2.514,211.289143,8.106,0.0,0.0,0.0,0.0,0.0,0.0,0,A,151315


## 7. Save to Parquet for Feast

In [14]:
# Save feature files
features_dir= Path("/shared/feature_repo/data")

os.makedirs(features_dir, exist_ok=True)

# Ray's offline store requires sorted data to avoid slow manual point-in-time joins
print('Sorting dataframes for efficient Ray merge_asof...')
sales_df = sales_df.sort_values(['store', 'dept', 'date'], ascending=True).reset_index(drop=True)
store_expanded_df = store_expanded_df.sort_values(['store', 'date'], ascending=True).reset_index(drop=True)
print('✓ Dataframes sorted by (store, dept, date) and (store, date)\n')

sales_df.to_parquet(features_dir / 'sales_features.parquet', index=False)
store_expanded_df.to_parquet(features_dir / 'store_features.parquet', index=False)

print('Saved feature files:')
for file in ['sales_features.parquet', 'store_features.parquet']:
    path =  features_dir/ file
    size_mb = path.stat().st_size / 1024 / 1024
    print(f'   ✓ {file}: {size_mb:.1f} MB')

print('\nFeature engineering complete!')
print(f'   Sales features: {len(sales_df):,} records')
print(f'   Store features: {len(store_expanded_df):,} records')

Sorting dataframes for efficient Ray merge_asof...
✓ Dataframes sorted by (store, dept, date) and (store, date)

Saved feature files:
   ✓ sales_features.parquet: 18.5 MB
   ✓ store_features.parquet: 0.4 MB

Feature engineering complete!
   Sales features: 421,570 records
   Store features: 606,242 records


## 8. Initialize Feast Feature Store

In [ ]:
!cp -r feature_repo/ /shared

In [17]:
store = FeatureStore(repo_path='/shared/feature_repo')

print('Feast initialized')
print(f'   Project: {store.project}')
print(f'   Offline store: {store.config.offline_store.type}')

Feast initialized
   Project: sales_demand_forecasting
   Offline store: file


## 9. Register Features with Feast

In [18]:
%%bash
cd /shared/feature_repo
feast apply

No project found in the repository. Using project name sales_demand_forecasting defined in feature_store.yaml
Applying changes for project sales_demand_forecasting


/opt/app-root/lib64/python3.12/site-packages/feast/feature_store.py:583: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(


Created project sales_demand_forecasting
Created entity dept
Created entity store
Created feature view sales_history_features
Created feature view store_external_features
Created on demand feature view temporal_transformations
Created on demand feature view feature_transformations
Created feature service demand_forecasting_service

Created sqlite table sales_demand_forecasting_sales_history_features
Created sqlite table sales_demand_forecasting_store_external_features



In [19]:
# Refresh store
store = FeatureStore(repo_path='/shared/feature_repo')

print('Registered Feature Views:')
for fv in store.list_feature_views():
    print(f'    {fv.name}: {len(fv.features)} features')

print('\n Feature Services:')
for fs in store.list_feature_services():
    print(f'    {fs.name}')

Registered Feature Views:
    sales_history_features: 8 features
    store_external_features: 13 features

 Feature Services:
    demand_forecasting_service


## 10. Test Feature Retrieval

In [20]:
# Sample 1000 records for testing
entity_df = sales_df[['store', 'dept', 'date']].sample(1000, random_state=42).copy()
entity_df = entity_df.rename(columns={'date': 'event_timestamp'})

print(f' Retrieving features from Feast offline store...')
print(f'   Entity rows: {len(entity_df):,}')

start = time.time()
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('demand_forecasting_service'),
).to_df()
elapsed = time.time() - start

print(f'\n Retrieved in {elapsed:.2f} seconds')
print(f'   Shape: {training_df.shape}')
print(f'   Features: {training_df.shape[1]}')

training_df.head(3)

 Retrieving features from Feast offline store...
   Entity rows: 1,000

 Retrieved in 2.93 seconds
   Shape: (1000, 31)
   Features: 31


,store,dept,event_timestamp,weekly_sales,is_holiday,sales_lag_1,sales_lag_2,sales_lag_4,sales_rolling_mean_4,sales_rolling_mean_12,...,has_markdown,store_type,store_size,sales_velocity,sales_acceleration,demand_stability_score,sales_normalized,temperature_normalized,sales_per_sqft,markdown_efficiency
0,31,67,2010-02-05 00:00:00+00:00,6939.56,False,NaN,NaN,NaN,6939.56,6939.56,...,0,A,203750,NaN,NaN,1.0,0.034698,0.358421,0.034059,6939.56
1,12,6,2010-02-05 00:00:00+00:00,5926.11,False,NaN,NaN,NaN,5926.11,5926.11,...,0,B,112238,NaN,NaN,1.0,0.029631,0.468105,0.052799,5926.11
2,28,29,2010-02-05 00:00:00+00:00,7473.91,False,NaN,NaN,NaN,7473.91,7473.91,...,0,A,206302,NaN,NaN,1.0,0.037370,0.468105,0.036228,7473.91


## 11. Summary

### Completed:
✅ Downloaded Walmart dataset (421K records)  
✅ Computed time-series features (lags, rolling stats)  
✅ Created store features (external factors, metadata)  
✅ Saved to Parquet (sales_features, store_features)  
✅ Registered with Feast (feature views, services)  
✅ Tested offline feature retrieval (25-32 features)

### Next Steps:
Proceed to **Notebook 02** to submit distributed training with Kubeflow Training SDK